# Assignment 4: Neo4j Graph Database

### Step 1 - Download and Install Neo4j Desktop, Set Up Project Database

In [20]:
!pip3 install tomlkit
!pip3 install -U neo4j

In [21]:
from neo4j import GraphDatabase

database_name = "apan5400"
username = "neo4j"
password = "apan5400"
uri = "bolt://localhost:7687/" + database_name

driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session()

print("Successfully connected to Neo4j!")

Successfully connected to Neo4j!


#### Delete All Nodes and Relationships

In [22]:
query_delete = ("MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r")
result_delete = session.run(query_delete)
print("All Nodes and relationships are deleted!")

All Nodes and relationships are deleted!


### Step 2 - Populate Graph Database

In [23]:
import os
import urllib

datapath = os.path.abspath("companies.csv")
print("file:///{0}".format(datapath))

file:////Users/sarahdoctor/Desktop/DataManagement/companies.csv


#### Import csv to neo4j database

In [24]:
companies = (
    "LOAD CSV WITH HEADERS FROM 'file:///{datapath}' AS line "
    "CREATE (company:Company {{name: line.company, employees: toInteger(line.employees), year: toInteger(line.founded), revenue: toFloat(line.revenue)}}) "
    
    "MERGE  (city:City {{name: line.city}}) "
    "MERGE  (state:State {{name: COALESCE(line.state, 'NA')}}) " 
    "MERGE  (country:Country {{name: line.country}}) "
    
    "CREATE (company)-[:HEADQUARTERED_IN]->(city) "
    "CREATE (city)-[:LOCATED_IN]->(state) "
    "CREATE (state)-[:PART_OF]->(country)"
).format(datapath=datapath)

result = session.run(companies)
print("All companies were imported from the CSV file!")

All companies were imported from the CSV file!


### Step 3 - Build and run Python Cypher queries against Neo4j:

#### 1) Which companies are located in the state of California (via 2nd degree relation)?

In [25]:
query1 = ("MATCH (state:State {name: 'California'})-[*1..2]-(company:Company) \
          RETURN DISTINCT company.name as companies")
result1 = session.run(query1)
answer1 = [record["companies"] for record in result1]

In [26]:
print('\nThe companies that are located in California (via 2nd degree relation):')
for company_name in answer1:
    print(company_name)


The companies that are located in California (via 2nd degree relation):
Apple
Alphabet


#### 2) Which companies have been founded in the last quarter of the 20th century [1975-2000]? 

In [27]:
query2 = (
    """
    MATCH (company:Company)
    WHERE company.year >= 1975 AND company.year <= 2000
    RETURN company.name, company.year
    ORDER BY company.year
    """
)
result2 = session.run(query2)
answer2 = [{"company.name": record["company.name"], 
            "company.year": record["company.year"]} for record in result2]

In [28]:
print("\nThe companies founded in the last quarter of the 20th century:")
for result in answer2:
    print(f"{result['company.name']}, founded in {result['company.year']}")


The companies founded in the last quarter of the 20th century:
Microsoft, founded in 1975
Apple, founded in 1976
China National Petroleum Corporation, founded in 1988
Amazon, founded in 1994
Alphabet, founded in 1998
Alibaba Group, founded in 1999


#### 3) What is the average annual revenue for all companies in the USA?

In [29]:
query3 = (
    """
    MATCH (company:Company)-[*1..3]-(country:Country {name: 'USA'})
    WITH DISTINCT company 
    RETURN avg(company.revenue) as avg_revenue
    """
)
result3 = session.run(query3)
answer3 = [record["avg_revenue"] for record in result3]

In [30]:
for result in answer3:
    print(f"\nThe average annual revenue for all USA companies is {result} billion")


The average annual revenue for all USA companies is 294.176 billion


#### 4) Which companies are headquartered in Japan (via 3rd degree relation)?

In [31]:
query4 = (
    """
    MATCH (country:Country {name: 'Japan'})-[*1..3]-(company:Company) 
    RETURN DISTINCT company.name as companies
    """
)
result4 = session.run(query4)
answer4 = [record["companies"] for record in result4]

In [32]:
print('\nThe companies that are headquartered in Japan (via 3rd degree relation):')
for company_name in answer4:
    print(company_name)


The companies that are headquartered in Japan (via 3rd degree relation):
Sony Group
Toyota Motor


#### 5) What is the total number of employees of all companies in the USA and Germany (via 3rd degree relation)?

In [33]:
query5 = (
    """
    MATCH (company:Company)-[*1..3]-(country:Country)
    WHERE country.name IN ['USA','Germany']
    WITH DISTINCT company
    RETURN sum(company.employees) as total_employees
    """
)

result5 = session.run(query5)
answer5 = [record["total_employees"] for record in result5]

In [34]:
for result in answer5:
    print(f"\nThe total number of employees for all USA and Germany companies is {result}")


The total number of employees for all USA and Germany companies is 2571925


#### Delete All Nodes and Relationships

In [36]:
query_delete = ("MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r")
result_delete = session.run(query_delete)
print("All Nodes and relationships are deleted!")

All Nodes and relationships are deleted!
